In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
import pickle

from typing import List, Tuple, Dict
import ffmpeg as ff


import warnings
warnings.filterwarnings(action='ignore') 

/home/ssrlab/anaconda3/envs/kyuwon_video_swin_transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_gt = pd.read_csv('./dataset/train.csv',encoding='EUC-KR')
test_gt = pd.read_csv('./dataset/test.csv', encoding='EUC-KR')

In [3]:
train_gt

,path,label
0,단어_106_가렵다_정면_1_이숙기.MOV,0
1,단어_108_가슴_정면_1_이숙기.MOV,1
2,단어_113_감전_정면_1_이숙기.MOV,2
3,단어_120_개_정면_1_이숙기.MOV,3
4,단어_121_거실_정면_1_이숙기.MOV,4
...,...,...
65,단어_127_계곡_정면_8.MOV,5
66,단어_152_기절_정면_8.MOV,6
67,단어_180_도둑_정면_8.MOV,7
68,단어_189_동전_정면_8.MOV,8


In [4]:
test_gt

,path,label
0,단어_106_가렵다_정면_9.MOV,0
1,단어_108_가슴_정면_9.MOV,1
2,단어_113_감전_정면_9.MOV,2
3,단어_120_개_정면_9.MOV,3
4,단어_121_거실_정면_9.MOV,4
5,단어_127_계곡_정면_9.MOV,5
6,단어_152_기절_정면_9.MOV,6
7,단어_180_도둑_정면_9.MOV,7
8,단어_189_동전_정면_9.MOV,8
9,단어_276_아빠_정면_9.MOV,9


## 각 영상마다 frame, width, height 확인

In [5]:
def check_preprocessing_input(file_path: str, file_name: str, dictionary: Dict[str, str], training: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    probe = ff.probe(filePath)
    video_streams = [stream for stream in probe["streams"] if stream["codec_type"] == "video"]
    print('frame = ', video_streams[0]['nb_frames'], 'width = ', video_streams[0]['coded_width'], ', height = ',video_streams[0]['coded_height'])
    del probe
    return

In [6]:
# training_set_data = []

path = './dataset/TRAIN'
for filename in os.listdir(path):
    filePath = path+'/'+filename
    print(filePath)
    check_preprocessing_input(file_path = filePath, file_name = filename, dictionary = train_gt)
    # training_set_data.append(preprocessing_input(file_path= filePath, file_name= filename, dictionary= gt, training= True))
    

./dataset/TRAIN/단어_106_가렵다_정면_1_이숙기.MOV
frame =  190 width =  1920 , height =  1088
./dataset/TRAIN/단어_113_감전_정면_8.MOV
frame =  332 width =  1920 , height =  1088
./dataset/TRAIN/단어_108_가슴_정면_6.MOV
frame =  205 width =  1920 , height =  1088
./dataset/TRAIN/단어_180_도둑_정면_1_이숙기.MOV
frame =  180 width =  1920 , height =  1088
./dataset/TRAIN/단어_108_가슴_정면_1_이숙기.MOV
frame =  177 width =  1920 , height =  1088
./dataset/TRAIN/단어_180_도둑_정면_8.MOV
frame =  329 width =  1920 , height =  1088
./dataset/TRAIN/단어_127_계곡_정면_1_이숙기.MOV
frame =  184 width =  1920 , height =  1088
./dataset/TRAIN/단어_120_개_정면_4.MOV
frame =  172 width =  1920 , height =  1088
./dataset/TRAIN/단어_108_가슴_정면_3.MOV
frame =  343 width =  1920 , height =  1088
./dataset/TRAIN/단어_113_감전_정면_6.MOV
frame =  228 width =  1920 , height =  1088
./dataset/TRAIN/단어_152_기절_정면_4.MOV
frame =  183 width =  1920 , height =  1088
./dataset/TRAIN/단어_127_계곡_정면_6.MOV
frame =  236 width =  1920 , height =  1088
./dataset/TRAIN/단어_120_개_정면_7.MOV
fr

# 추출 전략

##### 모든 영상에서 각각 30프레임씩 추출
 - (frame/30) 프레임 만큼 이동하면서 프레임 추출
 - width, height은 (224,224)로 resize

In [7]:
def get_number_of_frames(file_path: str) -> int:
    probe = ff.probe(filePath)
    video_streams = [stream for stream in probe["streams"] if stream["codec_type"] == "video"]
    #width = video_streams[0]['coded_width']
    #height = video_streams[0]['coded_height']
    del probe
    return video_streams[0]['nb_frames']

def extract_N_video_frames(file_path: str, number_of_samples: int = 6) -> List[np.ndarray]:
    nb_frames = int(get_number_of_frames(file_path= filePath))
    
    div = 30
    div_frames = nb_frames / div
    temp = 0
    video_frames = []
    print('이번 영상은')
    cap = cv2.VideoCapture(filePath)
    for ind in range(30):
        cap.set(1,int(temp))
        temp += div_frames
        res, frame = cap.read()
        video_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    print('개수 = ', len(video_frames))
    del cap
    return video_frames

def resize_image(image: np.ndarray, new_size: Tuple[int,int]) -> np.ndarray:
    return cv2.resize(image, new_size, interpolation = cv2.INTER_AREA)

def preprocessing_input(file_path: str, file_name: str, gt = train_gt, training: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    sampled = extract_N_video_frames(file_path= filePath, number_of_samples= 6)
    resized_images = [resize_image(image= im, new_size= (224,224)) for im in sampled]
    preprocessed_video = np.stack(resized_images)
    if sampled == None:
        print(file_name)
    for i in range(len(gt)):
        if gt.loc[i,'path'] == file_name:
            video_gt = gt.loc[i,'label']
    return (preprocessed_video, video_gt)

In [8]:
training_set_data = []

path = './dataset/TRAIN'
i=1
for filename in os.listdir(path):
    filePath = path+'/'+filename
    training_set_data.append(preprocessing_input(file_path= filePath, file_name= filename, gt= train_gt, training= True))
    print(i)
    i+=1

이번 영상은


개수 =  30
1
이번 영상은
개수 =  30
2
이번 영상은
개수 =  30
3
이번 영상은
개수 =  30
4
이번 영상은
개수 =  30
5
이번 영상은
개수 =  30
6
이번 영상은
개수 =  30
7
이번 영상은
개수 =  30
8
이번 영상은
개수 =  30
9
이번 영상은
개수 =  30
10
이번 영상은
개수 =  30
11
이번 영상은
개수 =  30
12
이번 영상은
개수 =  30
13
이번 영상은
개수 =  30
14
이번 영상은
개수 =  30
15
이번 영상은
개수 =  30
16
이번 영상은
개수 =  30
17
이번 영상은
개수 =  30
18
이번 영상은
개수 =  30
19
이번 영상은
개수 =  30
20
이번 영상은
개수 =  30
21
이번 영상은
개수 =  30
22
이번 영상은
개수 =  30
23
이번 영상은
개수 =  30
24
이번 영상은
개수 =  30
25
이번 영상은
개수 =  30
26
이번 영상은
개수 =  30
27
이번 영상은
개수 =  30
28
이번 영상은
개수 =  30
29
이번 영상은
개수 =  30
30
이번 영상은
개수 =  30
31
이번 영상은
개수 =  30
32
이번 영상은
개수 =  30
33
이번 영상은
개수 =  30
34
이번 영상은
개수 =  30
35
이번 영상은
개수 =  30
36
이번 영상은
개수 =  30
37
이번 영상은
개수 =  30
38
이번 영상은
개수 =  30
39
이번 영상은
개수 =  30
40
이번 영상은
개수 =  30
41
이번 영상은
개수 =  30
42
이번 영상은
개수 =  30
43
이번 영상은
개수 =  30
44
이번 영상은
개수 =  30
45
이번 영상은
개수 =  30
46
이번 영상은
개수 =  30
47
이번 영상은
개수 =  30
48
이번 영상은
개수 =  30
49
이번 영상은
개수 =  30
50
이번 영상은
개수 =  30
51
이번 영상은
개수 =  30
52
이번 영상은
개수 =  30
53
이번 영상은
개수

In [9]:
def reshape_to_expected_input(dataset: List[Tuple[np.ndarray, np.ndarray]]) -> Tuple[np.ndarray,np.ndarray]:
    
    x0_list = []
    y_list = []
    for i in range(0,len(dataset)):
        x0_list.append(dataset[i][0])
        y_list.append(dataset[i][1])
    return (np.stack(x0_list),np.stack(y_list))

In [10]:
train_input = reshape_to_expected_input(dataset= training_set_data)
del training_set_data

In [11]:
savename = 'training_set.dat'
with open(savename, "wb") as f:
    pickle.dump(train_input, f)

In [12]:
test_set_data = []

path = './dataset/TEST'
i=0
for filename in os.listdir(path):
    filePath = path+'/'+filename
    test_set_data.append(preprocessing_input(file_path= filePath, file_name= filename, gt= test_gt, training= False))
    print(i)
    i+=1

이번 영상은
개수 =  30
0
이번 영상은
개수 =  30
1
이번 영상은
개수 =  30
2
이번 영상은
개수 =  30
3
이번 영상은
개수 =  30
4
이번 영상은
개수 =  30
5
이번 영상은
개수 =  30
6
이번 영상은
개수 =  30
7
이번 영상은
개수 =  30
8
이번 영상은
개수 =  30
9
이번 영상은
개수 =  30
10
이번 영상은
개수 =  30
11
이번 영상은
개수 =  30
12
이번 영상은
개수 =  30
13
이번 영상은
개수 =  30
14
이번 영상은
개수 =  30
15
이번 영상은
개수 =  30
16
이번 영상은
개수 =  30
17
이번 영상은
개수 =  30
18
이번 영상은
개수 =  30
19


In [13]:
test_input = reshape_to_expected_input(dataset= test_set_data)
del test_set_data

In [14]:
savename = 'test_set.dat'
with open(savename, "wb") as f:
    pickle.dump(test_input, f)

# csv 파일 인코딩 확인

In [15]:
import chardet

with open('/home/ssrlab/kw/개인/Industry-Project/ai/dataset/train.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

In [16]:
print(result)

{'encoding': 'EUC-KR', 'confidence': 0.99, 'language': 'Korean'}
